# Compute Spectra for Events

This notebook processes event MAT files and computes spectra using multitaper method.

## Workflow
1. Load `eqinfo.mat` catalog
2. For each event, copy the base MAT file to a new directory
3. Call `update_data_spec()` to compute and save spectra

In [ ]:
import os
import sys
import shutil
import numpy as np
from scipy.io import loadmat
from joblib import Parallel, delayed

# Add src to path
sys.path.insert(0, os.path.abspath('../src'))
from update_data_spec import update_data_spec

## Configuration

In [ ]:
# Paths
eqfile = '../data/matfile/eqinfo.mat'
evdir = '../data/matfile'
evdir_target = evdir + '_spec'

# Create target directory
os.makedirs(evdir_target, exist_ok=True)

print(f"Source directory: {evdir}")
print(f"Target directory: {evdir_target}")

## Load Event Catalog

In [ ]:
# Load eqinfo
eqinfo_mat = loadmat(eqfile, squeeze_me=True, struct_as_record=False)
#eqinfo_mat = loadmat(eqfile)
eqinfo_origin = eqinfo_mat['eqinfo']
eqinfo = {}

# Convert to dict if needed
if not isinstance(eqinfo_origin, dict):
    for field in eqinfo_origin._fieldnames:
        value = getattr(eqinfo_origin, field)
        if not isinstance(value, np.ndarray):
            eqinfo[field] = np.array([value])
        else:
            eqinfo[field] = value

print(f"Loaded {len(eqinfo['id'])} events from {eqfile}")

## Process Events

Loop through all events, copy MAT files to target directory, and compute spectra using parallel processing.

In [ ]:
def calculate_spectra(eqinfo, event_idx, evdir, evdir_target):
    # Extract event info
    evid = str(int(eqinfo['id'][event_idx]))
    evyr = str(int(eqinfo['qyr'][event_idx]))
    evmon = f"{int(eqinfo['qmon'][event_idx]):02d}"
    
    # Build paths
    matfile0 = f"{evdir}/{evyr}/{evmon}/ev{evid}.mat"
    matpath = f"{evdir_target}/{evyr}/{evmon}"
    matfile = f"{matpath}/ev{evid}.mat"

    # Create target directory if needed
    os.makedirs(matpath, exist_ok=True)

    if os.path.exists(matfile0):
        # Copy file if it doesn't exist in target
        shutil.copyfile(matfile0, matfile)

        print(f"Processing event {evid} ...")
        update_data_spec(matfile, 0)

    else:
        print(f"{evid} does not exist! skipped.")

In [ ]:

num_events = len(eqinfo['id'])

n_cores = 8
parallel = Parallel(n_jobs=n_cores, backend='loky', verbose=5)

parallel(delayed(calculate_spectra)(eqinfo, i, evdir, evdir_target) for i in range(num_events))

print(f"\nDone! Processed {num_events} events.")